In [0]:
import numpy as np
import math
import os.path
import csv
import glob
import tensorflow as tf
import h5py as h5py
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model

In [0]:
# Package for face landmark detection
!pip install mlxtend
import mlxtend

In [0]:
import matplotlib.pyplot as plt
from mlxtend.image import extract_face_landmarks

# New Section

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
! ls '/content/drive/My Drive/CSCE_689'


 0.avi	  27.avi   6.avi				   model.h5
 10.avi   28.avi   7.avi				   model.json
 11.avi   29.avi   8.avi				   Test_1.avi
 12.avi   2.avi    9.avi				  'timeLabel: 31.json'
 13.avi   30.avi   combinedlabels.csv			  'timeLabel: 32.json'
 14.avi   31.avi   Data_60_10.npy			  'timeLabel: 33.json'
 15.avi   32.avi   Data.npy				  'timeLabel: 34.json'
 16.avi   33.avi   Features.csv				  'timeLabel: 35.json'
 17.avi   34.avi   Features_final.csv			  'timeLabel: 36.json'
 18.avi   35.avi   Features.npy				  'timeLabel: 37.json'
 19.avi   36.avi   Fold5_part2_features_60_10.csv.gsheet  'timeLabel: 38.json'
 1.avi	  37.avi   Fold5_part2_features_60_10.npy	  'timeLabel: 39.json'
 20.avi   38.avi  'Fold5_part2_labels_60_10 (1).gsheet'   'timeLabel: 40.json'
 21.avi   39.avi   Fold5_part2_labels_60_10.csv		  'timeLabel: 41.json'
 22.avi   3.avi    Fold5_part2_labels_60_10.gsheet	   timeLabel.json
 23.avi   40.avi   Fold5_part2_labels_60_10.npy		   TimePlot.json
 24.avi   41.avi   Labels.csv				 

In [1]:
cd '/content/drive/My Drive/CSCE_689/'

[Errno 2] No such file or directory: '/content/drive/My Drive/CSCE_689/'
/content


### Define and compute inputs terms 

In [0]:
def eye_aspect_ratio(eye):
	
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [0]:
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[14], mouth[18])
    C = distance.euclidean(mouth[12], mouth[16])
    mar = (A ) / (C)
    return mar

In [0]:
def circularity(eye):
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)

In [0]:
def mouth_over_eye(eye):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    mouth_eye = mar/ear
    return mouth_eye

In [0]:
def jaw_dist(jaw,nose):
  list= np.array([distance.euclidean(d, nose) for d in jaw])
  return list.max()/list.mean()



In [0]:
def getFrame(sec):
    start = 0
    vidcap.set(cv2.CAP_PROP_POS_MSEC, start + sec*1000)
    hasFrames,image = vidcap.read()
    return hasFrames, image

In [0]:
def get_landmarks(vidcap,i):
  sec = 0
  frameRate = 0.5
  success, image = getFrame(sec)
  count = 0
  data=[]
  labels=[]
  timestamp=[]
  while success and count < int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):
      landmarks = extract_face_landmarks(image)
      try:
          sum(sum(landmarks))
          if sum(sum(landmarks)) != 0:
              count += 1
              data.append(landmarks)
              labels.append([i])
              # cv2.imwrite("frame%d.jpg" % count, image)
              timestamp.append(sec)
              sec = sec + frameRate
              sec = round(sec, 3)
              success, image = getFrame(sec)

              # print('success: '+ str(success)+str(int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))))
          else:
              sec = sec + frameRate
              sec = round(sec, 3)
              success, image = getFrame(sec)
              print(str(i)+' not detected')
      except:
          # print('success: '+ str(success)+str(int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))))
          count=int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))
          pass
      # print(count)
  return data,labels,timestamp

In [0]:
# Generate Landmarks for all 41 training videos
from scipy.spatial import distance
import cv2
data = []
labels = []
frmes=[]
timestamp=[]
for i in range(42):
    vidcap = cv2.VideoCapture(str(i) + '.avi')
    data.extend(get_landmarks(vidcap,i)[0])
    labels.extend(get_landmarks(vidcap,i)[1])
    timestamp.extend(get_landmarks(vidcap,i)[2])
        

In [0]:
##Create Features
import numpy as np
import math
data = np.array(data)
features = []
main_list=[]
jaw_list=[]
for d in data:
    eye = d[36:68]
    nose=d[28]
    jaw=d[5:13]
    main_list.append(eye)
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    cir = circularity(eye)
    movt = jaw_dist(jaw,nose)
    mouth_eye = mouth_over_eye(eye)
    features.append([ear, mar, cir, mouth_eye,movt])

In [0]:
import pandas as pd
features = np.array(features)
features.shape
time= np.array(timestamp)
x=pd.DataFrame()
x['label']=labels
x['timestamp']=time

In [0]:
# Save files
np.save(open('Data.npy', 'wb'),data) ## Save landmark data
np.savetxt('Timestamps.csv',time)
np.savetxt("Features_final.csv", features, delimiter = ",")
np.savetxt("Labels.csv", labels, delimiter = ",")

In [0]:
## Save a combine file of label and timestamp. It will be converted to Labels_final.csv which will have labels of 0 (for not yawning) and 1 (for yawning)
x.to_csv("combinedlabels.csv") 

In [0]:
# Loading the files 
import numpy as np
import pandas as pd
labels=pd.read_csv('Labels_final.csv',header=None)
features=pd.read_csv('Features_final.csv',header=None)